# MNIST

The goal of this notebook is to manually build a neural network for the MNIST data set for digit recognition.

## Importing labraries and loading the data

In [11]:
import numpy as np
import pandas as pd
from scipy.special import expit
import sys
import matplotlib.pyplot as plt

In [8]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
X_train = df_train.drop(labels=["label"], axis=1)
y_train = df_train["label"]
X_test = df_test

In [62]:
# plot handwritten digits

fig, ax = plt.subplots(nrows=2, ncols=5, sharex=True, sharey=True)
ax = ax.flatten()

for i in range(10):
    img = X_train[i:i+1].reshape(28, 28)
    ax[i].imshow(img, cmap='Greys', interpolation='nearest')

ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.show()

AttributeError: 'DataFrame' object has no attribute 'reshape'

## Defining the neural network class

In [ ]:
class NeuralNetMLP(object):
    def __init__(self, n_output, n_features, n_hidden=30,
                l1=0.0, l2=0.0, epochs=500, eta=0.001,
                alpha=0.0)